In [1]:
!jupyter nbconvert --to=python Alk_bias_profile.ipynb

[NbConvertApp] Converting notebook Alk_bias_profile.ipynb to python
[NbConvertApp] Writing 2378 bytes to Alk_bias_profile.py


In [2]:
class Alk_bias_profile:   

    def __init__(self,runname,resultpath,savepath,mesh,ncfileAlk,ncfileDIC,
                 first_year,last_year,savefig=False, regional=False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncfileAlk = ncfileAlk
        self.ncfileDIC = ncfileDIC
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.regional = regional
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        
        from load_interp_GLODAP_python3 import GLODAPdata
        
        years = np.arange(first_year, last_year+1,1)
        #mesh       = pf.load_mesh(self.meshpath)
        meshdiag = pf.get_meshdiag(mesh)
        runid      =  self.runname
        
        unitsAlk = 'Alk Bias [mmol m$^{-3}$]'
        
        
        # load data -------------------------------------------------------------------------------------

        Alkfesom = pf.get_data(resultpath, "Alk", years, mesh,
                               how="mean", compute=True, runid=runid, silent=True)

        Alkglodap_input = GLODAPdata(runid,resultpath,mesh,ncfileAlk,'TAlk_mmol', get_overview=False)

        Alkglodap = Alkglodap_input.glodap_int
        Alkglodap[Alkfesom == 0] = 0
        
        # Load and derive profiles

        nod_area = np.ma.masked_equal(meshdiag.nod_area.values, 0)
        mask = pf.get_mask(mesh, "Global Ocean")

        Alkfesom_by_area = ((np.ma.masked_equal(Alkfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        Alkglodap_by_area = ((np.ma.masked_equal(Alkglodap[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        Alkfesom_weighted_Global = Alkfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        Alkglodap_weighted_Global = Alkglodap_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        
        ######
        Bias = Alkfesom_weighted_Global-Alkglodap_weighted_Global
        ######
        self.Bias_out = Bias